## Amazon Vine Review Analysis


**Project Overview**

**Objective:**
The main objective of this project is to analyze Amazon reviews written by members of the paid Amazon Vine program. The Amazon Vine program is a service that allows manufacturers and publishers to receive reviews for their products. This analysis aims to determine if there is any bias toward favorable reviews from Vine members.

**Methodology:**

1. **Data Collection:**
   - We utilized the power of big data to gather a comprehensive dataset of Amazon reviews.
   - The dataset was extracted using PySpark, a powerful tool for handling large-scale data processing.

2. **ETL Process:**
   - **Extract:** The dataset was extracted from Amazon's review database.
   - **Transform:** Data transformation was performed using PySpark to clean, filter, and prepare the data for analysis. This included handling missing values, normalizing text data, and categorizing reviews.
   - **Load:** The transformed data was then loaded into an AWS RDS (Relational Database Service) instance for secure and scalable storage. From there, it was further loaded into pgAdmin for detailed analysis.

3. **Data Analysis:**
   - We employed a combination of PySpark, Pandas, and SQL to conduct an in-depth analysis of the reviews.
   - Key metrics such as review ratings, review length, and the frequency of positive and negative words were analyzed.
   - Statistical tests and visualizations were used to identify patterns and trends in the data.

4. **Bias Detection:**
   - To determine if there is any bias toward favorable reviews from Vine members, we compared the distribution of review ratings between Vine and non-Vine members.
   - Sentiment analysis was performed to assess the overall sentiment of the reviews.
   - We also examined the correlation between review ratings and the presence of certain keywords or phrases.

**Results:**
The analysis provided insights into the nature of reviews written by Vine members compared to non-Vine members. Key findings included differences in average ratings, review length, and sentiment. These results helped to identify any potential bias in the reviews provided by Vine members.

**Conclusion:**
This project demonstrated the effectiveness of using big data tools like PySpark and Pandas for large-scale data analysis. By leveraging AWS RDS and pgAdmin, we ensured efficient data storage and management. The findings from this analysis can be used to improve the transparency and reliability of the Amazon Vine program.

In [25]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Install Postgres Driver
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Fetched 336 kB in 1s (240 kB/s)
Reading package lists... Done
-

In [26]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

--2023-02-11 04:27:56--  https://jdbc.postgresql.org/download/postgresql-42.5.0.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1046274 (1022K) [application/java-archive]
Saving to: ‘postgresql-42.5.0.jar.3’

postgresql-42.5.0.j 100%[===================>]   1022K  5.67MB/s    in 0.2s    

2023-02-11 04:27:57 (5.67 MB/s) - ‘postgresql-42.5.0.jar.3’ saved [1046274/1046274]



In [27]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.5.0.jar").getOrCreate()

Load Amazon Data into Spark DataFrame

In [28]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

# Show DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...|2015-08-31 00:00:00|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...| 

Create DataFrames

In [29]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
amazonReview_df = df.select(["review_id","customer_id","product_id","product_parent","product_title","review_date","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
amazonReview_df.show(5)

+--------------+-----------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+
|     review_id|customer_id|product_id|product_parent|       product_title|        review_date|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543|LG G4 Case Hard T...|2015-08-31 00:00:00|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650|Selfie Stick Fibl...|2015-08-31 00:00:00|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154|Tribe AB40 Water ...|2015-08-31 00:00:00|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704|RAVP

In [30]:
# Create the vine_table. DataFrame
vine_df = amazonReview_df.select(["review_id","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
|R11LOHEDYJALTN|          5|            0|          0|   N|                Y|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|                Y|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|                Y|
|R2L15IS24CX0LI|          5|            0|          0|   N|                Y|
|R1DJ8976WPWVZU|          3|            0|          0|   N|     

In [31]:
# create a new DataFrame where the total_votes count is equal to or greater than 20
vine_vote_df = vine_df.filter("total_votes >= 20")
vine_vote_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [32]:
# create a new DataFrame or table to retrieve all the rows 
# where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

helpfulVotes_df = vine_vote_df.withColumn('helpful_votes_%',vine_vote_df['helpful_votes']/vine_vote_df['total_votes'])
votePercent_df = helpfulVotes_df.filter(helpfulVotes_df["helpful_votes_%"]  >=.5).show()

#helpful_votes_df = vine_vote_df.filter((vine_vote_df["helpful_votes"] / vine_vote_df["total_votes"]) >= .5)
#helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   helpful_votes_%|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|0.9152542372881356|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|0.7142857142857143|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|0.9523809523809523|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|0.6363636363636364|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|0.9574468085106383|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|0.9423076923076923|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|0.6666666666666666|
|R2EE2TR4MRDV0U|          5|  

In [33]:
# create a new DataFrame or table that retrieves all the rows 
# where a review was written as part of the Vine program (paid), vine == 'Y'

paid_vine_df = vine_vote_df.filter(vine_vote_df["vine"]=="Y")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1MAOLI5FJHAFM|          4|          249|        261|   Y|                N|
| R9PYAUDIBJVEC|          4|           12|         22|   Y|                N|
|R2DOTPESCGMW6A|          4|            5|         24|   Y|                N|
| R6V9SHMMG5M8F|          5|          101|        110|   Y|                N|
|R37PVLT6ELL5J4|          4|          181|        209|   Y|                N|
|R1IG7IRDCIKJBN|          5|            9|         48|   Y|                N|
| R2FSFGWZF24V9|          4|           50|         51|   Y|                N|
|R3SRW1E8J56IGV|          5|          262|        281|   Y|                N|
| R86Z11D4CWOFM|          4|           32|         36|   Y|                N|
| RNP01HW9YISJO|          4|           20|         23|   Y|     

In [34]:
# create a new DataFrame or table that retrieves all the rows 
# where a review was written as part of the Vine program (paid), vine == 'N'

unpaid_vine_df = vine_vote_df.filter(vine_vote_df["vine"]=="N")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [35]:
# total number of reviews (paid vs unpaid)

paid_review_df = paid_vine_df.count()
paid_review_df 

618

In [36]:
unpaid_vine_df = unpaid_vine_df.count()
unpaid_vine_df 

71969

In [37]:
# number of 5-star reviews (paid vs unpaid)
paid_fiveStar_df = paid_vine_df.filter(paid_vine_df["star_rating"]==5).count()
paid_fiveStar_df

223

In [38]:
#unpaid_fiveStar_df = unpaid_vine_df.filter(unpaid_vine_df["star_rating"]== 5).count()
#unpaid_fiveStar_df

unpaid_fiveStar_df = vine_vote_df.filter((vine_vote_df["vine"]=="N") & (vine_vote_df["star_rating"]== 5))
unpaid_fiveStar_df.count()

31390

In [45]:
#1 the percentage of 5-star reviews for the two types of review (paid vs unpaid).
paid_fiveStar_percent_df = (paid_fiveStar_df/paid_review_df )*100
paid_fiveStar_percent_df

36.084142394822

In [46]:
#2 the percentage of 5-star reviews for the two types of review (paid vs unpaid).
unpaid_fiveStar_percent_df = (unpaid_fiveStar_df/unpaid_vine_df )*100
unpaid_fiveStar_percent_df

In [55]:
# Helpful vote counts (paid vs unpaid)
paid_helpful_df = votePercent_df.filter((votePercent_df["vine"]=="Y") & (votePercent_df["helpful_votes_%"]  >= .5)).count()
paid_helpful_df.count()


In [ ]:
unpaid_helpful_df = votePercent_df.filter((votePercent_df["vine"]=="N") & (votePercent_df["helpful_votes_%"]  >= .5)).count()
unpaid_helpful_df.count()